<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/eval_default_m5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import gc

PATH = '/kaggle/input/m5-forecasting-accuracy/'

TIME = 'evaluation'

if TIME == 'evaluation':
    file = 'sales_train_evaluation.csv'
    print('Need to changed other data................................')
else:
    file = 'sales_train_validation.csv'   
    
print(TIME, ' ', file)   

Need to changed other data................................
evaluation   sales_train_evaluation.csv


In [ ]:
#default reduce mem
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns: 
    col_type = df[col].dtypes
    if col_type in numerics: 
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)  
      else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [ ]:
train_df = reduce_mem_usage(pd.read_csv(PATH + file))
calendar_df = reduce_mem_usage(pd.read_csv(PATH + 'calendar.csv'))
price_df = reduce_mem_usage(pd.read_csv(PATH + 'sell_prices.csv'))

Mem. usage decreased to 96.13 Mb (78.8% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [ ]:
end_period = train_df.iloc[0, 6:].shape[0]
print(end_period)

1941


In [ ]:
col = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df =  pd.melt(train_df, id_vars=col, var_name='d', value_name='sales')

#make test
test_df = pd.DataFrame()
for i in range(end_period+1,end_period+29):
    temp = train_df[col]
    temp = temp.drop_duplicates()
    temp['d'] = f'd_{i}'
    temp['sales'] = np.nan
    test_df = pd.concat([test_df,temp])
    
df = pd.concat([df, test_df])
df = df.reset_index(drop=True)

del temp,test_df, train_df
gc.collect()

for i in col:
    df[i] =  df[i].astype('category')
    
#release day, drop 0 from data
release = price_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release.columns = ['store_id', 'item_id', 'release']
df = df.merge(release, on = ['store_id', 'item_id'], how = 'left')
df = df.merge(calendar_df[['wm_yr_wk', 'd']], on = ['d'], how = 'left')
df = df[df.wm_yr_wk >= df.release]
df = df.reset_index(drop=True)
df['release'] = df.release - df.release.min()
#after need convert to cat
df = reduce_mem_usage(df, verbose=True)

Mem. usage decreased to 1594.82 Mb (14.6% reduction)


In [ ]:
df['item_id'] = df['item_id'].astype('category')
df['store_id'] = df['store_id'].astype('category')
df.to_pickle('default_m5.pkl')
print('Saved')

Saved


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47735397 entries, 0 to 47735396
Data columns (total 10 columns):
 #   Column    Dtype   
---  ------    -----   
 0   id        category
 1   item_id   category
 2   dept_id   category
 3   cat_id    category
 4   store_id  category
 5   state_id  category
 6   d         object  
 7   sales     float16 
 8   release   int16   
 9   wm_yr_wk  int16   
dtypes: category(6), float16(1), int16(2), object(1)
memory usage: 1003.1+ MB


In [ ]:
default_col = list(df)

In [ ]:
price_df['price_max'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
price_df['price_min'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
price_df['price_std'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
price_df['price_mean'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')
price_df['count_price_un'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
price_df['count_item_un'] =price_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')
##Normalized Data, normalized = (x-min(x))/(max(x)-min(x))
price_df['norm'] = (price_df.sell_price - price_df.price_min)/(price_df.price_max - price_df.price_min) 
# price_df['diff_sellprice&max'] = price_df['price_max'] - price_df['sell_price']
# price_df['sem_rank'] = price_df.groupby(['store_id','item_id'])['sell_price'].transform('sem').rank(pct=True)
calendar_p = calendar_df[['wm_yr_wk','month','year']].drop_duplicates(subset= 'wm_yr_wk')
price_df = price_df.merge(calendar_p[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_p

price_df['price_momentum'] = price_df['sell_price']/price_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
price_df['price_momentum_w'] = price_df['sell_price']/price_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(7))
price_df['price_momentum_m'] = price_df['sell_price']/price_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
price_df['price_momentum_y'] = price_df['sell_price']/price_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')
del price_df['month'], price_df['year']
df = df.merge(price_df, on=['store_id','item_id','wm_yr_wk'], how='left')

In [ ]:
col = [i  for i in df.columns if i not in default_col]
df = reduce_mem_usage(df[col], verbose=True)
df.to_pickle('agg_prices.pkl')
print('Saved')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Mem. usage decreased to 1456.77 Mb (15.8% reduction)
Saved


In [ ]:
del df
gc.collect()

40


dif event by calendar

{'Eid al-Fitr', "Father's day", 'IndependenceDay'}

In [ ]:
# event_name = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay',
#        'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
#        'LentWeek2', 'Purim End', 'StPatricksDay', 'Easter', 'Pesach End',
#        'OrthodoxEaster', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
#        'NBAFinalsStart', 'NBAFinalsEnd', 'Ramadan starts', "Father's day",
#        'IndependenceDay', 'Eid al-Fitr']

In [ ]:
def between_dates(data, calendar):
  """
  default interval = +/-7 days
  -7days, event_name date + 7days  

  """
  event_name = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay',
       'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'Purim End', 'StPatricksDay', 'Easter', 'Pesach End',
       'OrthodoxEaster', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', 'Ramadan starts', "Father's day",
       'IndependenceDay', 'Eid al-Fitr']

  #zeros values not data
  drop_event = [
                'Cinco De Mayo_2016-05-05',
                'Mother\'s day_2016-05-08',
                'MemorialDay_2016-05-30',
                'NBAFinalsStart_2016-06-02',
                'NBAFinalsEnd_2016-06-19',
                'Ramadan starts_2016-06-07'
                ]

  for  ev in event_name:
    event_date = calendar[calendar.date >= '2015'][calendar.event_name_1 == ev].date
    for d in event_date.values:   
      start = str(pd.Timestamp(d) - pd.to_timedelta(7, unit='d')).split(' ')[0]
      end = str(pd.Timestamp(d) + pd.to_timedelta(7, unit='d')).split(' ')[0]
      out = data[data.date.between(start, end)].groupby('id')['sales'].mean()
      out.name = ev + '_' + d
      data = data.merge(out, how= 'left', on= 'id')

  print('Correct....')

  data.drop(drop_event, axis = 1, inplace = True)
  data = reduce_mem_usage(data, verbose=True)
  del calendar
  gc.collect()
  return data

data = between_dates(data, calendar_df)
data.to_pickle('diff_event_calendar.pkl')
print('Saved')

In [ ]:
df = pd.read_pickle('default_m5.pkl')

df = df[['id', 'd']]
col = [
    'date',
    'd',
    'event_name_1',
    'event_type_1',
    # 'event_name_2',
    # 'event_type_2',
    'snap_CA',
    'snap_TX',
    'snap_WI'
    ]

df = df.merge(calendar_df[col], on = 'd', how = 'left')
for c in col:
    df[c] = df[c].astype('category')


# Convert to DateTime
df['date'] = pd.to_datetime(df['date'])

def time_features(df):
    df.date = pd.to_datetime(df.date)
    attr = ['day',            
          # 'is_month_end',
          # 'is_month_start',
          # 'is_quarter_end',
          # 'is_quarter_start',
          # 'is_year_end',
          # 'is_year_start',
          'month',
          'quarter',
          'week',
          "dayofweek",    
          'year']
    
    for at in attr:
        dtype = np.int16 if at == "year" else np.int8
        df[at] = getattr(df.date.dt, at).astype(dtype)
    df["weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8) #Monday=0, Sunday=6

time_features(df)

# Remove date
del df['date']

df = reduce_mem_usage(df, verbose=True)
df.iloc[:, 2:].to_pickle('agg_days.pkl')
print('Saved')

Mem. usage decreased to 1139.68 Mb (0.0% reduction)
Saved


In [ ]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_pickle('default_m5.pkl')

df = df[['id', 'd']]
col = [   
    'd',
    'event_name_1',
    'event_type_1',
    'snap_CA',
    'snap_TX',
    'snap_WI'
    ]

df = df.merge(calendar_df[col], on = 'd', how = 'left')

def encode_categorical(df, cols):  
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        #not_null = df[col][df[col].notnull()]
        #df[col] = df[col].cat.add_categories('Unknown') #for calendar
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)
    return df

calendar = encode_categorical(
  df[['event_name_1', 'event_type_1']], ["event_name_1", "event_type_1"])

calendar = reduce_mem_usage(calendar, verbose=True)
calendar.to_pickle('event_name_LEncoder.pkl')
print('Save')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Mem. usage decreased to 455.24 Mb (58.3% reduction)
Save


In [ ]:
df = pd.read_pickle('default_m5.pkl')
df['d'] = df['d'].apply(lambda x: x[2:]).astype(np.int16)

del df['wm_yr_wk']
df.to_pickle('default_m5.pkl')
del df
print('Correct')

Correct


## add WMT

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pandas_datareader.data as web
import datetime as dt
%matplotlib inline

#Read Stock Data from Yahoo Finance
"""
Исторические данные варьируются от 2011-01-29 до 2016-06-19. Таким образом, продукты имеют (максимальную) продажу
история 1 941 2 дня / 5,4 года (данные испытаний h = 28 дней не включены).
"""
end = dt.datetime(2016, 9, 1)
#start = dt.datetime(end.year-2, end.month, end.day)
start = dt.datetime(2011, 1, 1)
df = web.DataReader("WMT", 'yahoo', start, end)
df = df.reset_index() 
df.columns = ['date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']

####plot

In [ ]:
train_period= df.iloc[:1373, :] #[df.Date == '2016-06-17'] #idx = 1373
test_period= df.iloc[1373:1373+28, :] #30 or 19 next month
test_period_after_test_month= df.iloc[1373+28:, :]

In [ ]:
#fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada')
# px.line(
#     x = df.Date,
#     y = df['Adj Close'],
#     title = 'Walmart',
# )
fig = make_subplots(rows = 2, cols = 1, shared_xaxes=True)
fig.add_trace(
    go.Scatter(
        x = df.date,
        y = df['Adj Close'],name ="All data Adj Close" ,
    ), row =1, col =1
)

fig.add_trace(
    go.Scatter(
        x = train_period.date,
        y = train_period['Adj Close'],name ="Train",
        marker=dict(color="black")
        ), row =2, col =1
)

fig.add_trace(
    go.Scatter(
        x = test_period.date,
        y = test_period['Adj Close'],name ="Test",
        marker=dict(color="red")
        ), row =2, col =1
)

fig.add_trace(
    go.Scatter(
        x = test_period_after_test_month.date,
        y = test_period_after_test_month['Adj Close'],name ="After test",
        marker=dict(color="green")
        ), row =2, col =1
)
# Add shapes
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0='2011-01-31',
            y0=65.1,
            x1='2016-09-01',
            y1=65.1,
            line=dict(
                color="Red",
                width=1
            )),  row =2, col =1)

#67.86
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0='2011-01-31',
            y0=67.9,
            x1='2016-09-01',
            y1=67.9,
            line=dict(
                color="Lightcoral",
                width=1
            )), row =2, col =1)

#2013-apr-7
fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0='2013-04-07',
            y0=35,
            x1='2013-04-07',
            y1=80,
            line=dict(
                color="DarkOrange",
                width=1
            )), row =2, col =1)

fig.update_xaxes(title_text="Years/Days", row=2, col=1)
fig.update_yaxes(title_text= 'Price', mirror=True)
fig.update_layout(height=800, width=1000, title_text="Walmart Inc")

### change data

In [ ]:
df.date = df.date.astype('str')

In [ ]:
data.date = data.date.astype('str')

In [ ]:
df = df[['date','Close','Volume', 'Adj Close']]

In [ ]:
data = data.merge(df, how='left', on='date')

In [ ]:
#1884 '2016-03-27',1912 '2016-04-24'
#<1884 train, 1884 -1912 valid
#test 1912 >

data.to_pickle('stocks_wmt.pkl')
print('Saved')

#lags

In [ ]:
# import time
# df = pd.read_pickle(PATH_AGG)
# df = df[['id', 'd', 'sales']]


# # Rollings
# # with sliding shift
# for d_shift in [1, 5, 7, 10, 14, 19, 24]: 
#     print('Shifting period:', d_shift)
#     for d_window in [7, 14, 21, 30, 37, 44, 51, 60]:
#         col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
#         df[col_name] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
# print('%0.2f min: Lags' % ((time.time() - start_time) / 60))
# df = reduce_mem_usage(df, verbose=True)
# df.iloc[:, 3:].to_pickle('changed_lags_m5.pkl')
# print('Saved')

In [ ]:
import time
df = pd.read_pickle(PATH_AGG)
df = df[['id', 'd', 'sales']]


# Rollings
# with sliding shift
for d_shift in [1, 5, 7]: #, 10, 14, 19]: 
    print('Shifting period:', d_shift)
    for d_window in [7, 14, 21, 30, 37, 44, 51, 60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        df[col_name] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(d_shift).ewm(span = d_window, adjust=False)\
                                    .mean()).astype(np.float16)
    
    
df = reduce_mem_usage(df, verbose=True)
df.iloc[:, 3:].to_pickle('v1_changed_lags_m5.pkl')
print('Saved')

Shifting period: 1
Shifting period: 5
Shifting period: 7
Mem. usage decreased to 2459.78 Mb (0.0% reduction)
Saved


#trend

I drop by mem

In [ ]:
df= pd.read_pickle(PATH_D1 + 'default_m5.pkl')     

###v1

In [ ]:
df['d_avg_sold_state'] = df.groupby(['item_id','state_id','d'])['sales'].transform('mean').astype(np.float16)
df['avg_sold'] = df.groupby(['item_id', 'state_id'])['sales'].transform('mean').astype(np.float16)
df['trend_item_state'] = (df['d_avg_sold_state'] - df['avg_sold']).astype(np.float16)

###v2

In [ ]:
df['d_avg_sold'] = df.groupby(['item_id','d'])['sales'].transform('mean').astype(np.float16)
df['avg_sold'] = df.groupby(['item_id'])['sales'].transform('mean').astype(np.float16)
df['trend_item_id'] = (df['d_avg_sold'] - df['avg_sold']).astype(np.float16)

In [ ]:
df = df.drop(['d_avg_sold','avg_sold'], axis =1)

In [ ]:
df[['trend_item_state', 'trend_item_id', 'd_avg_sold_state']].to_pickle('trends.pkl')

#prob

I drop by mem

In [ ]:
df = pd.concat([
                pd.read_pickle('default_m5.pkl'),
                pd.read_pickle('agg_prices.pkl'),
                pd.read_pickle('agg_days.pkl')],
                axis =1)[['id', 'year', 'month', 'day','sales']]

In [ ]:
for i in [2011, 2012,2013,2014,2015,2016]:
    y = df[df.year==i].groupby(['id', 'year', 'month', 'day'])['sales'].sum()
    print(i)
    if i == 2011:
        temp = df.merge(y.unstack(level=2).mean(axis =1).rename("diff"), how = 'left', on= ['id', 'year', 'day'])
        del y
        gc.collect()
    else:
        temp = temp.merge(y.unstack(level=2).mean(axis =1).rename("diff"), how = 'left', on= ['id', 'year', 'day'])
        del y
        gc.collect()   

In [ ]:
temp.iloc[:, 5:].sum(axis= 1).to_frame(name = 'prob_sales').to_pickle('prob_sales.pkl')